# Bedrock Client Testing Notebook

This notebook tests the `invoke_model` method in the Bedrock client (`genaiic-idp-accelerator/lib/idp_common_pkg/idp_common/bedrock/client.py`). It includes tests for:

1. Claude models with different parameter combinations
2. Nova models with different parameter combinations
3. Parameter validation and error handling

Each test verifies that parameters are correctly passed to the Bedrock API.

## 1. Setup and Imports

First, we'll import the necessary libraries and set up logging to see debug output.

In [ ]:
# Let's make sure that modules are autoreloaded
%load_ext autoreload
%autoreload 2

ROOTDIR="../.."
# First uninstall existing package (to ensure we get the latest version)
%pip uninstall -y idp_common

# Install the IDP common package with all components in development mode
%pip install -q -e "{ROOTDIR}/lib/idp_common_pkg[dev, all]"

# Note: We can also install specific components like:
# %pip install -q -e "{ROOTDIR}/lib/idp_common_pkg[ocr,classification,extraction,evaluation]"

# Check installed version
%pip show idp_common | grep -E "Version|Location"

# Optionally use a .env file for environment variables
try:
    from dotenv import load_dotenv
    load_dotenv()  
except ImportError:
    pass  

Found existing installation: idp_common 0.2.19
Uninstalling idp_common-0.2.19:
  Successfully uninstalled idp_common-0.2.19
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Version: 0.2.19
Location: /Users/miislamg/miniconda3/envs/genaiic-idp-accelerator/lib/python3.13/site-packages
Note: you may need to restart the kernel to use updated packages.


In [2]:
import sys
import os
import json
import logging
import boto3
from typing import Dict, Any, List, Optional, Union

# Import the Bedrock client
from idp_common.bedrock.client import BedrockClient, invoke_model

# Set up logging to see debug output
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

# Create a logger for this notebook
logger = logging.getLogger('bedrock_client_test')

### Helper Functions

Let's create some helper functions to make testing easier.

In [3]:
def print_response_summary(response: Dict[str, Any]) -> None:
    """Print a summary of the Bedrock response."""
    if not response:
        print("Empty response")
        return
        
    # Extract the actual response from the wrapper
    bedrock_response = response.get("response", {})
    
    # Print the output text
    if "output" in bedrock_response and "message" in bedrock_response["output"]:
        message = bedrock_response["output"]["message"]
        if "content" in message and isinstance(message["content"], list) and len(message["content"]) > 0:
            text = message["content"][0].get("text", "")
            print(f"Response text (truncated): {text[:200]}...")
    
    # Print usage information
    if "usage" in bedrock_response:
        usage = bedrock_response["usage"]
        print(f"Usage: {usage}")
    
    # Print metering information
    if "metering" in response:
        print(f"Metering: {response['metering']}")

## 2. Test Cases for Claude Models

Now we'll test the `invoke_model` method with a Claude model using different parameter combinations.

In [4]:
# Create a new Bedrock client instance
claude_client = BedrockClient(region="us-west-2")

# Define the Claude model ID
claude_model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"

# Define a simple system prompt
system_prompt = "You are a helpful assistant that provides concise answers."

# Define a simple user message
content = [{"text": "What is Amazon Bedrock?"}]

### Claude Test with Multiple Parameters

Let's test with multiple parameters: temperature, top_p, top_k, and max_tokens.

In [5]:
try:
    print("Testing Claude model with multiple parameters...")
    response = claude_client.invoke_model(
        model_id=claude_model_id,
        system_prompt=system_prompt,
        content=content,
        temperature=0.8,
        top_p=0.9,
        top_k=40,
        max_tokens=100
    )
    
    print("\nResponse summary:")
    print_response_summary(response)
    
except Exception as e:
    print(f"Error: {str(e)}")

Testing Claude model with multiple parameters...


2025-05-09 02:33:57,047 - idp_common.bedrock.client - INFO - Bedrock request attempt 1/8:
2025-05-09 02:33:58,805 - idp_common.bedrock.client - INFO - Response: {'ResponseMetadata': {'RequestId': '19b87967-8866-4e4f-a15d-42697edab9e3', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 09 May 2025 09:33:58 GMT', 'content-type': 'application/json', 'content-length': '569', 'connection': 'keep-alive', 'x-amzn-requestid': '19b87967-8866-4e4f-a15d-42697edab9e3'}, 'RetryAttempts': 0}, 'output': {'message': {'role': 'assistant', 'content': [{'text': 'Amazon Bedrock is a fully managed service by AWS that provides access to high-performing foundation models from various AI companies through a single API. It allows developers to build and scale generative AI applications without having to manage the underlying infrastructure.'}]}}, 'stopReason': 'end_turn', 'usage': {'inputTokens': 25, 'outputTokens': 51, 'totalTokens': 76, 'cacheReadInputTokens': 0, 'cacheWriteInputTokens': 0}, 'metrics': {'


Response summary:
Response text (truncated): Amazon Bedrock is a fully managed service by AWS that provides access to high-performing foundation models from various AI companies through a single API. It allows developers to build and scale gener...
Usage: {'inputTokens': 25, 'outputTokens': 51, 'totalTokens': 76, 'cacheReadInputTokens': 0, 'cacheWriteInputTokens': 0}
Metering: {'bedrock/us.anthropic.claude-3-7-sonnet-20250219-v1:0': {'inputTokens': 25, 'outputTokens': 51, 'totalTokens': 76, 'cacheReadInputTokens': 0, 'cacheWriteInputTokens': 0}}


## 3. Test Cases for Nova Models

Now we'll test the `invoke_model` method with a Nova model using different parameter combinations.

In [6]:
# Create a new Bedrock client instance
nova_client = BedrockClient(region="us-west-2")

# Define the Nova model ID
nova_model_id = "us.amazon.nova-pro-v1:0"

# Define a simple system prompt
nova_system_prompt = "You are a helpful assistant that provides concise answers."

# Define a simple user message
nova_content = [{"text": "What is Amazon Bedrock?"}]

### Nova Test with Multiple Parameters

Now let's test with multiple parameters: temperature, top_p, top_k, and max_tokens.

In [7]:
try:
    print("Testing Nova model with multiple parameters...")
    response = nova_client.invoke_model(
        model_id=nova_model_id,
        system_prompt=nova_system_prompt,
        content=nova_content,
        temperature=0.8,
        top_p=0.9,
        top_k=40,
        max_tokens=100
    )
    
    print("\nResponse summary:")
    print_response_summary(response)
    
except Exception as e:
    print(f"Error: {str(e)}")

2025-05-09 02:33:59,552 - idp_common.bedrock.client - INFO - Bedrock request attempt 1/8:


Testing Nova model with multiple parameters...


2025-05-09 02:34:00,398 - idp_common.bedrock.client - INFO - Response: {'ResponseMetadata': {'RequestId': '61ebd327-d396-4aaf-b9ce-a73e61f46eb3', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 09 May 2025 09:34:00 GMT', 'content-type': 'application/json', 'content-length': '323', 'connection': 'keep-alive', 'x-amzn-requestid': '61ebd327-d396-4aaf-b9ce-a73e61f46eb3'}, 'RetryAttempts': 0}, 'output': {'message': {'role': 'assistant', 'content': [{'text': 'Amazon Bedrock is a service that simplifies building and scaling generative AI applications using foundation models from leading AI companies.'}]}}, 'stopReason': 'end_turn', 'usage': {'inputTokens': 16, 'outputTokens': 22, 'totalTokens': 38}, 'metrics': {'latencyMs': 759}}



Response summary:
Response text (truncated): Amazon Bedrock is a service that simplifies building and scaling generative AI applications using foundation models from leading AI companies....
Usage: {'inputTokens': 16, 'outputTokens': 22, 'totalTokens': 38}
Metering: {'bedrock/us.amazon.nova-pro-v1:0': {'inputTokens': 16, 'outputTokens': 22, 'totalTokens': 38}}
